<a href="https://colab.research.google.com/github/sadikul1500/Movie-Recommender/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files

In [4]:
import pandas as pd

In [5]:
print('select movies data file')
uploaded = files.upload()

select movies data file


Saving movies.csv to movies (1).csv


In [6]:
for f in uploaded.keys():
  movies = pd.read_csv(f, usecols=['movieId', 'title'], dtype={'title':'str'})
  print(movies.head())


   movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)


In [7]:
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
dtypes: int64(1), object(1)
memory usage: 152.3+ KB
None


In [8]:
print('select user ratings data file')
uploaded = files.upload()

select user ratings data file


Saving ratings.csv to ratings (1).csv


In [9]:
for f in uploaded.keys():
  ratings = pd.read_csv(f, usecols=['userId', 'movieId', 'rating'])

In [10]:
print(ratings.head())
print(ratings.info())

   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB
None


In [11]:
array_ratings = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(1e-8)
print(array_ratings.head())
#print(array_ratings[0])

userId            1             2    ...           609           610
movieId                              ...                            
1        4.000000e+00  1.000000e-08  ...  3.000000e+00  5.000000e+00
2        1.000000e-08  1.000000e-08  ...  1.000000e-08  1.000000e-08
3        4.000000e+00  1.000000e-08  ...  1.000000e-08  1.000000e-08
4        1.000000e-08  1.000000e-08  ...  1.000000e-08  1.000000e-08
5        1.000000e-08  1.000000e-08  ...  1.000000e-08  1.000000e-08

[5 rows x 610 columns]


In [12]:
print(type(array_ratings))
array_ratings_np = array_ratings.to_numpy()
print(len(array_ratings_np))

<class 'pandas.core.frame.DataFrame'>
9724


In [ ]:
print(movies.loc[90]['title'])

Mr. Wrong (1996)


In [15]:
def getPrediction(distance, k):
    output_values = [dist[-1] for dist in distance[:k]]
    #prediction = max(set(output_values), key=output_values.count)
    return output_values #prediction

In [ ]:
print(array_ratings_np[0][0])

4.0


In [13]:
def getDistance(movieIndex, train): #, train
    #test = test[:len(test)-1]
    distance = []

    j = 0
    for training in train:
        x = 0
        for i in range(len(training)):
            x += (float(train[movieIndex][i]) - float(training[i])) ** 2
        #print(training)
        movie = movies.loc[j]['title'] #movies[movies['movieId'] == j+1].iloc[0]['title']
        distance.append([x ** .5, movie])
        j += 1

    return distance


In [14]:
def evaluateAlgorithm(test, k, movieIndex):
    
    distance = getDistance(movieIndex, array_ratings_np) #, train
    distance.sort()

    for i in range(k+1):
      print('i+1 ', end='')
      print(distance[i][1], end='')
      print(' with a distance of ', end='')
      print(distance[i][0])
    
    #prediction = getPrediction(distance, k)
    #print(prediction)
       
    

In [15]:

#test = input('enter a movie name with release year (e.g. Toy Story (1995)) ')
movieName = input('enter a movie name (eg Die Hard 2) ') #string.capwords()
year = input('enter release year of that movie (eg 1990) ')

test = movieName + ' (' + year + ')'

#movieId = movies[movies['movieId'] == test].iloc[0]['movieid']



enter a movie name (eg Die Hard 2) tfd
enter release year of that movie (eg 1990) 9879


In [2]:
import string
s = string.capwords('The BOSS of baby')
print(s)

The Boss Of Baby


In [9]:
#print(movies.index[movies['title'] == 'Kids (1995)'].tolist()[0])

147


In [17]:
import sys
try:
  movieIndex = movies.index[movies['title'] == test].tolist()[0] #movies[movies['title'] == test].to_numpy()
except:
  print('not found in the dataaset.\ncorrect spelling or release year')
  exit(0)

k = int(input('expected number of recommendations '))

print(movieIndex)
evaluateAlgorithm(test, k, movieIndex)

KeyboardInterrupt: ignored

In [ ]:
if __name__ == '__main__':
    #fileName = 'G:\\5 th semester\\dbms2\\knn\\iris.txt'
    #oFile = open('outIris.txt', 'w')
    #woFile = open('weight.txt', 'w')
    #moFile = open('mWeight.txt', 'w')

    #file = open(fileName, 'r')
    #lines = file.readlines()
    #numberOfLines = len(lines)
    #print(numberOfLines)
    '''
    data = []
    i = 0
    for line in lines:
        values = line.split(',')
        data.append(values)
        i += 1

    shuffle(data)
    shuffledData = list.copy(data)
    #print(data)

    numberOfFold = int(input("number of fold: "))
    '''
    k = int(input('value of k: '))
    percent = int(len(shuffledData) / numberOfFold)
    accuracy = []
    wAccuracy = []
    mAccuracy = []

    for i in range(numberOfFold):
        start = i * percent
        end = start + percent
        test = shuffledData[start : end]
        train = shuffledData[: start] + shuffledData[end-1:]
        #print(test)
        #print(train)
        neutral, weight, mWeight = evaluateAlgorithm(train, test, k, oFile, woFile, moFile)
        accuracy.append(neutral)
        wAccuracy.append(weight)
        mAccuracy.append(mWeight)

    print(accuracy)
    meanAccuracy = sum(accuracy) / len(accuracy)
    print('mean accuracy : ', meanAccuracy)

    print(wAccuracy)
    w_meanAccuracy = sum(wAccuracy) / len(wAccuracy)
    print('weighted mean accuracy : ', w_meanAccuracy)

    print(mAccuracy)
    m_meanAccuracy = sum(mAccuracy) / len(mAccuracy)
    print('weighted mean accuracy : ', m_meanAccuracy)

    file.close()
    oFile.close()
    woFile.close()
    moFile.close()